In [1]:
import pandas as pd
import numpy as np
from scipy import io

In [2]:
posters = io.loadmat("../data/posters.mat")

In [3]:
posters["523162"].shape

(92, 140, 3)

In [57]:
movies = pd.read_csv("../data/tmdb_api_data.csv", lineterminator="\n")
movies.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/bOGkgRGdhrBYJSLpXaxhXVstddV.jpg,"[12, 28, 14]",299536,en,Avengers: Infinity War,As the Avengers and their allies have continue...,153.811,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,2018-04-25,Avengers: Infinity War,False,8.3,12490
1,False,/5zfVNTrkhMu673zma6qhFzG01ig.jpg,[878],300668,en,Annihilation,"A biologist signs up for a dangerous, secret e...",29.516,/d3qcpfNwbAMCNqWDHzPQsUYiUgS.jpg,2018-02-22,Annihilation,False,6.3,4232
2,False,/zjG95oDnBcFKMPgBEmmuNVOMC90.jpg,"[35, 18]",299782,en,The Other Side of the Wind,"Surrounded by fans and skeptics, grizzled dire...",6.820,/kFky1paYEfHxfCYByEc9g7gn6Zk.jpg,2018-11-02,The Other Side of the Wind,False,7.1,55
3,False,/q9hnJ9SzwcF30seRtXEzLd5l1gw.jpg,"[18, 35, 14]",351044,en,Welcome to Marwen,When a devastating attack shatters Mark Hoganc...,61.973,/o45VIAUYDcVCGuzd43l8Sr5Dfti.jpg,2018-12-21,Welcome to Marwen,False,6.6,174
4,False,/AmO8I38bkHwKhgxPNrd6djBQyPU.jpg,"[53, 9648, 27, 14]",361292,en,Suspiria,A darkness swirls at the center of a world-ren...,41.461,/dzWTnkert9EoiPWldWJ15dnfAFl.jpg,2018-10-11,Suspiria,False,7.2,579


In [58]:
movies = movies.dropna(subset=["id", "poster_path", "vote_average"])
movies["rating"] = np.round(movies["vote_average"])
movies.shape

(7353, 15)

In [62]:
all_ids = [str(i) for i in movies["id"] if str(i) in posters and posters[str(i)].shape == (92, 140, 3)]

In [63]:
len(np.unique(all_ids)) - len(all_ids)

-2137

In [64]:
ids = np.unique(all_ids)

In [65]:
X = np.stack([posters[i] for i in ids])

movies_by_id = movies.set_index("id")
y = np.stack([int(movies_by_id.loc[int(i),"rating"].iloc[0]) if type(movies_by_id.loc[int(i),"rating"]) \
              == pd.Series else int(movies_by_id.loc[int(i),"rating"]) for i in ids])

In [66]:
X.shape

(5123, 92, 140, 3)

In [67]:
y.shape

(5123,)

In [68]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)

In [69]:
from sklearn.model_selection import train_test_split

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y)

In [71]:
# Importing all necessary libraries 
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras import backend as K 

In [75]:
# train_data_dir = 'v_data/train'
# validation_data_dir = 'v_data/test'
# nb_train_samples =400
# nb_validation_samples = 100
epochs = 10
batch_size = 16
img_size = (92, 140)
input_shape = img_size + (3,)

In [80]:
def get_model():
    model = Sequential() 
    model.add(Conv2D(32, (2, 2), input_shape=input_shape)) 
    model.add(Activation('relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 

    model.add(Conv2D(32, (2, 2))) 
    model.add(Activation('relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 

    model.add(Conv2D(64, (2, 2))) 
    model.add(Activation('relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 

    model.add(Flatten()) 
    model.add(Dense(64)) 
    model.add(Activation('relu')) 
    model.add(Dropout(0.5)) 
    model.add(Dense(dummy_y.shape[1])) 
    model.add(Activation('sigmoid'))
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model


In [82]:
estimator = KerasClassifier(build_fn=get_model, epochs=5, batch_size=5, verbose=0)

In [83]:
kfold = KFold(n_splits=10, shuffle=True)

In [84]:
%%time
results = cross_val_score(estimator, X_train, y_train, cv=kfold, n_jobs=-1)

CPU times: user 163 ms, sys: 227 ms, total: 389 ms
Wall time: 15min 58s


In [85]:
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 64.47% (3.01%)


In [86]:
estimator.predict(X_test)

AttributeError: 'KerasClassifier' object has no attribute 'model'